In [1]:
import tensorflow as tf 
import transformers # pre-trained transformer models (BERT)
import numpy as np
import pandas as pd
import nltk

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Ensure NLTK data is downloaded
def setup_nltk():
    """Download required NLTK data packages."""
    try:
        required_packages = ['punkt', 'stopwords']
        for package in required_packages:
            try:
                nltk.data.find(f'tokenizers/{package}')
            except LookupError:
                print(f"Downloading {package}...")
                nltk.download(package, quiet=True)
        return True
    except Exception as e:
        print(f"Error setting up NLTK: {str(e)}")
        return False

In [2]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data for BERT processing."""

    def __init__(
            self,
            sentence_pairs,
            labels=None,
            batch_size=32,
            shuffle=True,
            include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.sentence_pairs) / self.batch_size))

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_sentence_pairs = self.sentence_pairs[batch_indexes]

        encoded = self.tokenizer.batch_encode_plus(
            batch_sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_tensors="tf",
        )

        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[batch_indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [3]:
def simple_tokenize(text):
    """Simple word tokenization without relying on NLTK's punkt."""
    # Basic cleaning
    text = text.lower().strip()
    # Split on whitespace
    words = text.split()
    return words


def calculate_similarity_score(teacher_answer, student_answer, tokenizer, model):
    """Calculate cosine similarity between teacher's and student's answers using BERT embeddings."""
    # Tokenize and get embeddings
    def get_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Use [CLS] token embedding

    teacher_embedding = get_embedding(teacher_answer)
    student_embedding = get_embedding(student_answer)

    # Calculate cosine similarity
    similarity = cosine_similarity([teacher_embedding], [student_embedding])[0][0]
    return similarity

def calculate_length_score(teacher_answer, student_answer):
    """Simple length comparison between teacher's and student's answers."""
    teacher_len = len(teacher_answer.split())
    student_len = len(student_answer.split())
    
    # Avoid division by zero
    if teacher_len == 0:
        return 0

    # Allow some flexibility in length, but penalize very short answers
    length_ratio = min(student_len / teacher_len, 1)
    return length_ratio


In [4]:
def process_answers(teacher_file, student_file, total_test_marks):
    """Process answers from Excel files and calculate scores using BERT from Hugging Face."""
    try:
        # Load BERT model and tokenizer
        print("Loading BERT model...")
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        model.eval()  # Set model to evaluation mode

        # Read Excel files into DataFrames
        print("Reading Excel files...")
        teacher_df = pd.read_excel(teacher_file)
        student_df = pd.read_excel(student_file)

        # Validate required columns
        if teacher_df.shape[1] < 3:
            raise ValueError("Teacher Excel file must have at least 3 columns: Question, Answer, Total Marks.")
        if student_df.shape[1] < 2:
            raise ValueError("Student Excel file must have at least 2 columns: Question, Answer.")

        # Create dictionaries for quick lookup
        teacher_dict = {str(row[0]).strip(): str(row[1]).strip() for idx, row in teacher_df.iterrows()}
        marks_dict = {str(row[0]).strip(): float(row[2]) for idx, row in teacher_df.iterrows()}
        student_dict = {str(row[0]).strip(): str(row[1]).strip() for idx, row in student_df.iterrows()}

        # Check if questions match
        if set(teacher_dict.keys()) != set(student_dict.keys()):
            missing_in_student = set(teacher_dict.keys()) - set(student_dict.keys())
            missing_in_teacher = set(student_dict.keys()) - set(teacher_dict.keys())
            error_msg = ""
            if missing_in_student:
                error_msg += f"Questions missing in student file: {missing_in_student}\n"
            if missing_in_teacher:
                error_msg += f"Questions missing in teacher file: {missing_in_teacher}\n"
            if error_msg:
                raise ValueError(error_msg)

        # Initialize total marks and obtained marks
        total_marks_allotted = sum(marks_dict.values())
        if total_marks_allotted == 0:
            raise ValueError("Total marks allotted from teacher file is zero.")

        if total_test_marks != total_marks_allotted:
            print(f"Warning: The sum of marks per question ({total_marks_allotted}) does not match the total test marks ({total_test_marks}). Proceeding with normalization.")

        # Initialize results
        results = []
        total_obtained_marks = 0

        print("Calculating scores for each question...")

        for question in teacher_dict:
            teacher_answer = teacher_dict[question]
            student_answer = student_dict[question]
            marks_allotted = marks_dict[question]

            # Calculate similarity using BERT
            similarity_score = calculate_similarity_score(teacher_answer, student_answer, tokenizer, model)
            length_score = calculate_length_score(teacher_answer, student_answer)

            # Final score for the question
            question_score = similarity_score * length_score * marks_allotted

            # Accumulate total obtained marks
            total_obtained_marks += question_score

            # Append individual question results
            results.append({
                "question": question,
                "teacher_answer": teacher_answer,
                "student_answer": student_answer,
                "similarity_score": round(similarity_score * 100, 2),  # Percentage
                "length_score": round(length_score, 2),
                "marks_allotted": marks_allotted,
                "question_score": round(question_score, 2)
            })

        # Normalize total obtained marks to the total test marks
        normalization_factor = total_test_marks / total_marks_allotted
        final_score = int(total_obtained_marks * normalization_factor)

        return {
            "results_per_question": results,
            "total_obtained_marks": round(total_obtained_marks, 2),
            "total_marks_allotted": total_marks_allotted,
            "total_test_marks": total_test_marks,
            "final_score": final_score
        }

    except Exception as e:
        return {"error": str(e)}


In [ ]:
if __name__ == "__main__":
    print("Starting answer evaluation...")

    # Prompt for total marks of the test.
    while True:
        try:
            total_test_marks = float(input("Enter the total marks for the test: "))
            if total_test_marks <= 0:  # negative or zero inputs.
                print("Total marks must be a positive number. Please try again.")
                continue
            break
        except ValueError:  # non-numeric inputs.
            print("Invalid input. Please enter a numeric value for total marks.")
            
    student_path = 'C:/Users/suury/Desktop/Mini Project Sem5/Interface/student_answers.xlsx'
    teacher_path = 'C:/Users/suury/Desktop/Mini Project Sem5/Interface/teacher_answers.xlsx'
    
    # Process the Excel files.
    results = process_answers(teacher_path, student_path, total_test_marks)

    if "error" in results:
        print(f"\nError: {results['error']}")
    else:
        print("\nAnswer Evaluation Results:")
        print("-" * 50)
        for idx, res in enumerate(results["results_per_question"], start=1):
            print(f"Question {idx}: {res['question']}")
            print(f"Teacher's Answer: {res['teacher_answer']}")
            print(f"Student's Answer: {res['student_answer']}")
            print("\nScores:")
            # Adjusted the print statements to use the single similarity score.
            print(f"  Similarity Score: {res['similarity_score']}%")
            print(f"  Length Score: {res['length_score']:.2f}")
            print(f"  Marks Allotted: {res['marks_allotted']}")
            print(f"  Question Score: {res['question_score']}/{res['marks_allotted']}\n")
            print("-" * 50)

        print("Summary:")
        print(f"Total Obtained Marks: {results['total_obtained_marks']}/{results['total_marks_allotted']}")
        print(f"Total Test Marks: {results['total_test_marks']}")
        print(f"Final Score: {results['final_score']}/{results['total_test_marks']}")